In [ ]:

%pip -q install sagemaker==2.219.0 boto3 scikit-learn==1.5.1 pandas

import sagemaker, boto3, os, json, pandas as pd
from sagemaker.sklearn import SKLearn
from sagemaker import get_execution_role


session = sagemaker.Session()
bucket = session.default_bucket()
role = get_execution_role()  
prefix = "bank-marketing-no-container"

In [ ]:

BANK_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional-full.csv"

df = pd.read_csv(BANK_URL, sep=";")
df.to_csv("train.csv", index=False)


train_s3 = session.upload_data("train.csv", bucket=bucket, key_prefix=f"{prefix}/input/train")
print("Training data uploaded to:", train_s3)

In [ ]:
estimator = SKLearn(
    entry_point="train.py",        
    source_dir="../src",           
    role=role,
    instance_type="ml.m5.large",
    framework_version="1.2-1",
    py_version="py3",
    sagemaker_session=session,
    hyperparameters={"target-col": "y"},   
)
estimator.fit({"train": train_s3})

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    entry_point="inference.py",   
    source_dir="../src",          
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

In [ ]:
sample = {
    "age": 30,
    "job": "blue-collar",
    "marital": "married",
    "education": "secondary",
    "default": "no",
    "housing": "yes",
    "loan": "no",
    "contact": "cellular",
    "month": "may",
    "day_of_week": "mon",
    "duration": 300,
    "campaign": 1,
    "pdays": 999,
    "previous": 0,
    "poutcome": "nonexistent",
    "emp.var.rate": -1.8,
    "cons.price.idx": 92.893,
    "cons.conf.idx": -46.2,
    "euribor3m": 1.299,
    "nr.employed": 5099.1
}
print("Sample prediction:", predictor.predict({"features": sample}))

In [ ]:
predictor.delete_endpoint()